In [ ]:
import time

start_time = time.time()

In [ ]:
import cv2
import json
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import random

import torch
print(torch.cuda.is_available()) 

In [ ]:
# Init model & video
model = YOLO("YOLOv12mv8.pt").to("cuda") 
cap = cv2.VideoCapture(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's\machine cover.mp4")

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
frame_index = 0

In [ ]:
# Init DeepSORT
tracker = DeepSort(max_age=30)

In [ ]:
CONFIDENCE_THRESHOLD = 0.6  # Pas dit aan naar wens

# Opslagstructuur
object_tracks = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    timestamp = round(frame_index / fps, 2)
    results = model(frame)

    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = box.conf[0].item()
            if conf < CONFIDENCE_THRESHOLD:
                continue  # Sla lage confidence detecties over

            x1, y1, x2, y2 = box.xyxy[0].tolist()
            class_id = int(box.cls[0].item())
            class_name = model.names[class_id]

            # Voeg alleen toe als confidence >= threshold
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, class_name))

    # Voer tracking uit
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = str(track.track_id)
        ltrb = track.to_ltrb()  # [x1, y1, x2, y2]
        x, y, w, h = ltrb[0], ltrb[1], ltrb[2] - ltrb[0], ltrb[3] - ltrb[1]
        label = track.get_det_class()  # class_name

        if track_id not in object_tracks:
            object_tracks[track_id] = {
                "class_name": label,
                "first_seen": timestamp,
                "last_seen": timestamp,
                "trajectory": []
            }

        object_tracks[track_id]["last_seen"] = timestamp
        object_tracks[track_id]["trajectory"].append({
            "time": timestamp,
            "bbox": {
                "x": round(x, 2),
                "y": round(y, 2),
                "width": round(w, 2),
                "height": round(h, 2)
            }
        })

    frame_index += 1

cap.release()

In [ ]:
# Opslaan naar JSON
with open("detections_with_tracking_YOLOv12mv8_machine_cover_2.json", "w") as f:
    json.dump(object_tracks, f, indent=4)

print("Trackingresultaten opgeslagen in detections_with_tracking.json")

In [ ]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Totaal uitgevoerde tijd: {elapsed_time:.2f} seconden")

In [ ]:
# Load tracking JSON
with open("detections_with_tracking_YOLOv12mv8_stabilizer_pressure_control.json", "r") as f:
    tracking_data = json.load(f)

# Zet alles per frame-tijd
time_map = {}
for obj_id, data in tracking_data.items():
    for entry in data["trajectory"]:
        t = round(entry["time"], 2)
        if t not in time_map:
            time_map[t] = []
        box = entry["bbox"]
        time_map[t].append({
            "id": obj_id,
            "class_name": data["class_name"],
            "bbox": box
        })

# Open video
cap = cv2.VideoCapture(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's\stabilizer pressure control.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("video_with_boxes_YOLOv12mv8_stabilizer_pressure_control.mp4", fourcc, fps, (width, height))

frame_idx = 0
colors = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_time = round(frame_idx / fps, 2)
    detections = time_map.get(current_time, [])

    for det in detections:
        track_id = str(det["id"])
        class_name = det["class_name"]
        bbox = det["bbox"]
        x, y, w, h = int(bbox["x"]), int(bbox["y"]), int(bbox["width"]), int(bbox["height"])
        x2, y2 = x + w, y + h

        if track_id not in colors:
            colors[track_id] = [random.randint(0, 255) for _ in range(3)]
        color = colors[track_id]

        cv2.rectangle(frame, (x, y), (x2, y2), color, 2)
        cv2.putText(frame, f"{class_name} #{track_id}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()
print("Annotated video opgeslagen als video_with_boxes.mp4")

# Versie 2

In [ ]:
from ultralytics import YOLO

model = YOLO("YOLOv12mv8.pt")
results = model.track(
    source=r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's\stabilizer pressure control.mp4",  # <-- pas aan
    tracker="botsort.yaml",
    save=False,       # we maken straks zelf een video
    show=False,
    conf=0.6,
    persist=True,
    stream=True       # <- belangrijk! dit geeft ons de resultaten per frame
)

In [ ]:
import json

object_tracks = {}
frame_index = 0
fps = 30  # of haal het uit cv2.VideoCapture als je exact wilt zijn

for result in results:
    timestamp = round(frame_index / fps, 2)
    if result.boxes is None:
        frame_index += 1
        continue

    for i in range(len(result.boxes)):
        box = result.boxes[i]
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        track_id = str(int(box.id[0].item())) if box.id is not None else "untracked"
        class_id = int(box.cls[0].item())
        class_name = model.names[class_id]

        if track_id not in object_tracks:
            object_tracks[track_id] = {
                "class_name": class_name,
                "first_seen": timestamp,
                "last_seen": timestamp,
                "trajectory": []
            }

        object_tracks[track_id]["last_seen"] = timestamp
        object_tracks[track_id]["trajectory"].append({
            "time": timestamp,
            "bbox": {
                "x": round(x1, 2),
                "y": round(y1, 2),
                "width": round(x2 - x1, 2),
                "height": round(y2 - y1, 2)
            }
        })

    frame_index += 1

# Opslaan als JSON
with open("botsort_tracking_output.json", "w") as f:
    json.dump(object_tracks, f, indent=4)

print("JSON opgeslagen als botsort_tracking_output.json")


In [ ]:
import cv2
import random
import json

# Herlaad JSON
with open("botsort_tracking_output.json", "r") as f:
    tracking_data = json.load(f)

# Tijd-map per frame
time_map = {}
for obj_id, data in tracking_data.items():
    for entry in data["trajectory"]:
        t = round(entry["time"], 2)
        if t not in time_map:
            time_map[t] = []
        time_map[t].append({
            "id": obj_id,
            "class_name": data["class_name"],
            "bbox": entry["bbox"]
        })

# Open originele video
cap = cv2.VideoCapture(r"C:\Users\maxim\OneDrive\01-Opleidingen\03-MAAI\Afstuderen\Data\Video's\stabilizer pressure control.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("botsort_video_with_boxes.mp4", fourcc, fps, (width, height))

frame_idx = 0
colors = {}

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_time = round(frame_idx / fps, 2)
    detections = time_map.get(current_time, [])

    for det in detections:
        track_id = str(det["id"])
        class_name = det["class_name"]
        bbox = det["bbox"]
        x, y, w, h = int(bbox["x"]), int(bbox["y"]), int(bbox["width"]), int(bbox["height"])
        x2, y2 = x + w, y + h

        if track_id not in colors:
            colors[track_id] = [random.randint(0, 255) for _ in range(3)]
        color = colors[track_id]

        cv2.rectangle(frame, (x, y), (x2, y2), color, 2)
        cv2.putText(frame, f"{class_name} #{track_id}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)
    frame_idx += 1

cap.release()
out.release()

print("Annotated video opgeslagen als botsort_video_with_boxes.mp4")
